In [1]:
import numpy as np
import torch

In [2]:
from models import MPS, ComplexTensor

In [3]:
from tools import generate_binary_space

In [10]:
from qtools import pauli_exp

In [4]:
L=2
local_dim=2
bond_dim=1

In [5]:
basis = torch.tensor(generate_binary_space(L), dtype=torch.long)

In [6]:
psi = MPS(L=L, local_dim=local_dim, bond_dim=bond_dim)

In [7]:
psi.tensors

[ComplexTensor shape torch.Size([2, 1, 1]),
 ComplexTensor shape torch.Size([2, 1, 1])]

In [8]:
for i in range(L):
    psi.tensors[i].real.data = torch.tensor([1.,0]).view(2,1,1)
    psi.tensors[i].imag.data = torch.zeros(2).view(2,1,1)

In [9]:
with torch.no_grad():
    for b in basis:
        print(b," has amplitude:", psi.amplitude_normalized(b).display())

tensor([1, 1])  has amplitude: [0.+0.j]
tensor([1, 0])  has amplitude: [0.+0.j]
tensor([0, 1])  has amplitude: [0.+0.j]
tensor([0, 0])  has amplitude: [1.+0.j]


In [16]:
theta = (np.pi/2) * torch.ones(1,2)
# phi = torch.zeros(1,2)
phi = theta
U = pauli_exp(theta,phi)

In [17]:
with torch.no_grad():
    for b in basis:
        print(b," has amplitude:", psi.amplitude_normalized(b,rotation=U).display())

tensor([1, 1])  has amplitude: [0.-0.49999994j]
tensor([1, 0])  has amplitude: [-0.49999994+0.j]
tensor([0, 1])  has amplitude: [-0.49999994+0.j]
tensor([0, 0])  has amplitude: [0.+0.49999994j]
